In [1]:
import os
%pwd

'c:\\Users\\dwiat\\MLOps Courses\\e2e-data-science-project\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\dwiat\\MLOps Courses\\e2e-data-science-project'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/edatha/e2e-data-science-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="edatha"
os.environ["MLFLOW_TRACKING_PASSWORD"]="e84b4f536dbdb4f8b46a0fc25c0fcf86389f77ea"

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json


In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file_name = config.metric_file_name,
            target_column=schema.name,
            all_params=params,
            mlflow_uri="https://dagshub.com/edatha/e2e-data-science-project.mlflow"
        )
        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config=config
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse":rmse, "mae":mae, "r2":r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # Please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry/html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
                
                
                

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-03-07 15:07:14,314: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-07 15:07:14,319: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-07 15:07:14,324: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-07 15:07:14,329: INFO: common: Created directory at: artifacts]
[2025-03-07 15:07:14,333: INFO: common: Created directory at: artifacts/model_evaluation]
[2025-03-07 15:07:15,564: INFO: common: JSON file saved at: artifacts\model_evaluation\metrics.json]


2025/03/07 15:07:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticNetModel'.
2025/03/07 15:07:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 1
Created version '1' of model 'ElasticNetModel'.


🏃 View run spiffy-stork-629 at: https://dagshub.com/edatha/e2e-data-science-project.mlflow/#/experiments/0/runs/00206df7bf504927ae338fd1d279fbc6
🧪 View experiment at: https://dagshub.com/edatha/e2e-data-science-project.mlflow/#/experiments/0
